# Métrica 2_3_03 — Congruencia entre ingresos de la pareja y/o dependientes e ingresos totales declarados
| Escenario                                                              | Condición      | Resultado        |
| ---------------------------------------------------------------------- | -------------- | ---------------- |
| Sin pareja ni dependientes con actividad laboral (no obligatoria)      | —              | ⚪ **SIN_DATO**   |
| Pareja/dependiente con actividad laboral y se reportan ingresos > 0    | ✅              | 🟢 **CUMPLE**    |
| Pareja/dependiente con actividad laboral pero ingresos = 0 o faltantes | ❌              | 🔴 **NO_CUMPLE** |
| Sin datos suficientes                                                  | ⚪ **SIN_DATO** |                  |


In [1]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "2_3_03_CONGRUENCIA_INGRESOS_PAREJA_DEPENDIENTES"

DICT_INFO = {
    "pareja_publico_path": "declaracion.situacionPatrimonial.datosPareja.actividadLaboralSectorPublico",
    "pareja_privado_path": "declaracion.situacionPatrimonial.datosPareja.actividadLaboralSectorPrivadoOtro",
    "dependientes_path": "declaracion.situacionPatrimonial.datosDependientesEconomicos.dependienteEconomico",
    "ingresos_path": "declaracion.situacionPatrimonial.ingresos.ingresoMensualNetoParejaDependiente.valor",
    "obligatorio": False
}

# --- Funciones auxiliares ---

def extraer_lista(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p, {})
            else:
                return []
        if isinstance(actual, list):
            return actual
        elif isinstance(actual, dict):
            return [actual]
        return []
    except Exception:
        return []

def hay_actividad_laboral_pareja(doc):
    pareja_pub = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosPareja", {}).get("actividadLaboralSectorPublico")
    pareja_priv = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosPareja", {}).get("actividadLaboralSectorPrivadoOtro")
    return bool(pareja_pub) or bool(pareja_priv)

def dependiente_tiene_actividad(dep):
    try:
        return bool(dep.get("actividadLaboralSectorPublico")) or bool(dep.get("actividadLaboralSectorPrivadoOtro"))
    except Exception:
        return False

def existen_dependientes_activos(dependientes):
    if not isinstance(dependientes, list):
        return False
    return any(dependiente_tiene_actividad(dep) for dep in dependientes)

def obtener_ingreso(doc):
    try:
        valor = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("ingresos", {}).get("ingresoMensualNetoParejaDependiente", {}).get("valor", 0)
        if isinstance(valor, (int, float)):
            return valor
        return float(valor) if valor else 0
    except Exception:
        return 0

# --- Evaluación principal ---

def evaluar_metrica(doc):
    pareja_activa = hay_actividad_laboral_pareja(doc)
    dependientes = extraer_lista(doc, DICT_INFO["dependientes_path"])
    dependiente_activo = existen_dependientes_activos(dependientes)
    ingreso = obtener_ingreso(doc)

    # Sin pareja ni dependientes activos
    if not pareja_activa and not dependiente_activo and not DICT_INFO["obligatorio"]:
        return "SIN_DATO"

    # Pareja o dependiente activo con ingreso reportado
    if (pareja_activa or dependiente_activo) and ingreso > 0:
        return "CUMPLE"

    # Pareja o dependiente activo pero sin ingreso
    if (pareja_activa or dependiente_activo) and ingreso == 0:
        return "NO_CUMPLE"

    return "SIN_DATO"

# --- Procesamiento MongoDB ---

def procesar_metrica_2_3_03():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0}
    operaciones = []
    total = 0

    try:
        print(f"Procesando métrica {METRIC_ID}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosPareja": 1,
            "declaracion.situacionPatrimonial.datosDependientesEconomicos": 1,
            "declaracion.situacionPatrimonial.ingresos": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1
            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"Error en doc {doc.get('_id')}: {e}")

            resultados[resultado] = resultados.get(resultado, 0) + 1
            operaciones.append(UpdateOne({"_id": doc["_id"]}, {"$set": {METRIC_ID: resultado}}, upsert=True))

            if len(operaciones) >= 1000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f"  > Procesados {total} documentos...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Total procesados: {total}")
        for k, v in resultados.items():
            print(f"  > {k}: {v}")

    except Exception:
        traceback.print_exc()
    finally:
        try:
            client.close()
        except:
            pass
        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_2_3_03()


Procesando métrica 2_3_03_CONGRUENCIA_INGRESOS_PAREJA_DEPENDIENTES...


c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


  > Procesados 1000 documentos...
  > Procesados 2000 documentos...
  > Procesados 3000 documentos...
  > Procesados 4000 documentos...
  > Procesados 5000 documentos...
  > Procesados 6000 documentos...
  > Procesados 7000 documentos...
  > Procesados 8000 documentos...
  > Procesados 9000 documentos...
  > Procesados 10000 documentos...
  > Procesados 11000 documentos...
  > Procesados 12000 documentos...
  > Procesados 13000 documentos...
  > Procesados 14000 documentos...
  > Procesados 15000 documentos...
  > Procesados 16000 documentos...
  > Procesados 17000 documentos...
  > Procesados 18000 documentos...
  > Procesados 19000 documentos...
  > Procesados 20000 documentos...
Conexión cerrada.


KeyboardInterrupt: 